# HUD Fair Market Rents - Quick Start Guide

This notebook demonstrates how to use the **HUDFMRConnector** to access HUD Fair Market Rent data and perform affordability analysis.

**Data Sources:**
- Fair Market Rents (FMR) by bedroom count (0BR-4BR)
- Income limits (very low, low, median)
- Small Area FMRs (ZIP code level)

**Use Cases:**
- Housing affordability analysis
- Rental market comparisons
- Income qualification calculations

**API Access:** Requires HUD USER API key (free registration)

---

*© 2025 KR-Labs. Licensed under Apache-2.0.*

## 1. Setup and Import

In [ ]:
from krl_data_connectors.housing import HUDFMRConnector
import pandas as pd
import os

# Initialize connector with API key
# Get your API key from: https://www.huduser.gov/portal/dataset/fmr-api.html
api_key = os.getenv('HUD_API_KEY', 'your_api_key_here')
hud = HUDFMRConnector(api_key=api_key)

print("✓ HUD FMR connector initialized")

## 2. Get State FMRs via API

Retrieve Fair Market Rents for a state.

In [ ]:
# Get Rhode Island FMRs for 2023
ri_fmrs = hud.get_state_fmrs('RI', year=2023)

print(f"Retrieved FMR data for {len(ri_fmrs)} areas")
print("\nColumns:", ri_fmrs.columns.tolist())
ri_fmrs.head()

## 3. Load FMR Data from File

Alternative: Load downloaded FMR data from CSV/Excel.

In [ ]:
# If you prefer to use downloaded files
# Download from: https://www.huduser.gov/portal/datasets/fmr.html

# fmr_file = "path/to/FY2023_FMRs.xlsx"
# fmr_data = hud.load_fmr_data(fmr_file)
# print(f"Loaded {len(fmr_data)} FMR records")

## 4. Filter by Bedroom Count

Extract FMRs for specific bedroom counts.

In [ ]:
# Get 2-bedroom FMRs
fmr_2br = hud.get_fmr_by_bedrooms(ri_fmrs, bedroom_count=2)

print("2-Bedroom Fair Market Rents:")
print(fmr_2br[['countyname', 'metro_name', 'fmr_2br']].head())

# Summary statistics
print(f"\nAverage 2BR FMR: ${fmr_2br['fmr_2br'].mean():.0f}")
print(f"Min: ${fmr_2br['fmr_2br'].min():.0f}")
print(f"Max: ${fmr_2br['fmr_2br'].max():.0f}")

## 5. Calculate Affordability

Determine if housing is affordable based on income (30% rule).

In [ ]:
# Example: Household earning $50,000/year
household_income = 50000
typical_fmr = 1200  # 2BR FMR

affordability = hud.calculate_affordability(
    household_income=household_income,
    fmr_value=typical_fmr,
    income_threshold=0.30
)

print(f"Household Income: ${household_income:,}")
print(f"FMR: ${typical_fmr}")
print(f"\nMax Affordable Rent: ${affordability['max_affordable_rent']:.2f}")
print(f"Rent-to-Income Ratio: {affordability['rent_to_income_ratio']:.1%}")
print(f"Affordable: {affordability['affordable']}")
print(f"Monthly Surplus/Deficit: ${affordability['surplus_deficit']:.2f}")

## 6. Compare Counties

Analyze FMR differences across counties.

In [ ]:
# Compare Rhode Island counties
counties = ['Providence County', 'Newport County', 'Washington County']
comparison = hud.compare_fmrs(ri_fmrs, counties, bedroom_count=2)

print("County Comparison (2BR FMRs):")
print(comparison[['countyname', 'fmr_2br']].sort_values('fmr_2br', ascending=False))

## 7. Compare All Bedroom Sizes

View FMRs across all bedroom counts.

In [ ]:
# Get all bedroom FMRs for Providence County
providence = hud.get_county_fmrs(ri_fmrs, 'Providence County')

if not providence.empty:
    print("Providence County FMRs:")
    bedroom_cols = ['fmr_0br', 'fmr_1br', 'fmr_2br', 'fmr_3br', 'fmr_4br']
    for col in bedroom_cols:
        if col in providence.columns:
            bedrooms = col.split('_')[1].upper()
            value = providence[col].iloc[0]
            print(f"  {bedrooms}: ${value:.0f}")

## 8. Year-over-Year Analysis

Calculate FMR changes between years.

In [ ]:
# Get FMRs for two consecutive years
fmr_2023 = hud.get_state_fmrs('RI', year=2023)
fmr_2022 = hud.get_state_fmrs('RI', year=2022)

# Calculate year-over-year change for 2BR
yoy_change = hud.calculate_yoy_change(fmr_2023, fmr_2022, 'fmr_2br')

print("Year-over-Year FMR Changes (2BR):")
print(yoy_change[['countyname', 'fmr_2br_current', 'fmr_2br_previous', 
                   'yoy_change', 'yoy_change_pct']].round(2))

## 9. Affordability Across Income Levels

Analyze affordability for different income brackets.

In [ ]:
# Test multiple income levels
income_levels = [30000, 40000, 50000, 60000, 75000]
fmr_value = 1200  # 2BR FMR

print(f"Affordability Analysis (FMR: ${fmr_value}):\n")
for income in income_levels:
    result = hud.calculate_affordability(income, fmr_value)
    status = "✓ Affordable" if result['affordable'] else "✗ Not Affordable"
    print(f"${income:,}: {status} (Max: ${result['max_affordable_rent']:.0f})")

## 10. Export Results

Save FMR data for further analysis.

In [ ]:
# Export state FMRs
hud.export_to_csv(ri_fmrs, 'ri_fmrs_2023.csv')

# Export comparison
if not comparison.empty:
    hud.export_to_csv(comparison, 'ri_county_comparison.csv')

print("✓ Data exported successfully")

## Next Steps

**Explore More:**
- Filter by metro area: `get_metro_fmrs()`
- Get income limits: `get_income_limits()`
- Compare across states
- Analyze Small Area FMRs (ZIP level)

**Resources:**
- [HUD USER API Documentation](https://www.huduser.gov/portal/dataset/fmr-api.html)
- [FMR Documentation](https://www.huduser.gov/portal/datasets/fmr.html)
- [Income Limits](https://www.huduser.gov/portal/datasets/il.html)

**Use Cases:**
- Housing voucher program analysis
- Affordable housing site selection
- Regional cost-of-living comparisons
- Policy impact assessment